In [62]:
import pandas as pd
import numpy as np
import scipy as sci
from matplotlib import pyplot as plt

from ase import Atoms

from ase.visualize import view
from ase.io import read
from ase.io import write
from ase.build import molecule
from ase import units
import os

In [63]:
#import the geometry from CONCAR file CONCAR
name_file = 'molecule_library/CONTCAR_methylenebenzene_3x3'

#use ASE to read file and open as atoms object
atoms = read(name_file)
#view(atoms)

atoms_H = atoms[np.array(list(atoms.symbols))=='H']
atoms_C = atoms[np.array(list(atoms.symbols))=='C']

#visualize the adsorbed moiety
#view(atoms_H+atoms_C)

atoms_ads= atoms_H+atoms_C
atoms_ads.set_pbc(True)

In [64]:
#turn atoms_ads into dataframe
df_atoms = pd.DataFrame(list(atoms_ads.symbols),columns=['atom'])

df_pos = pd.DataFrame(atoms_ads.get_positions(),columns=['x','y','z'])
df = pd.concat([df_atoms,df_pos], axis=1)

# get center of position
COP = [df['x'].mean(),df['y'].mean(),df['z'].mean()]

In [65]:
#split into matrices consisting of coordinates of each element and shift to zero COP
df_H = df[df['atom']=='H']
df_C = df[df['atom']=='C']

#matricies with atomic coordinates shifted to the COP for C and H atoms with units of angstrom
M_H = df_H[['x','y','z']].to_numpy()-COP
M_C = df_C[['x','y','z']].to_numpy()-COP

In [66]:
#determine the vdW volume of all of the atoms without accounting for their overlap

#Cs vdw volume
vdW_R_C = 1.7 #radius, Ang
vdW_V_C = np.pi*4/3*vdW_R_C**3 #vol of single C

#Hs vdw volume
vdW_R_H = 1.2 #radius, Ang
vdW_V_H = np.pi*4/3*vdW_R_H**3 #vol of single C

In [67]:
# build a 3D grid and populate it with a sphere with the vdW volume
# for each atom expaned about its coordinates 

# add 2 above and below the maximum position of the structure in x, y, and z directions
# to ensure that the entire vdW spheres are included
c_mins = np.array([np.min(df['x']),np.min(df['y']),np.min(df['z'])-2])-2
c_maxs = np.array([np.max(df['x']),np.max(df['y']),np.max(df['z'])+2])+2
c_ranges = c_maxs-c_mins

#grid cube dimension
d = 0.01 #discrete size

#matrix dimensions calculated by normalizing the coordinate ranges by the cube dimension
M_size = np.ceil(c_ranges/d)
M_size = (np.ceil(M_size/2))*2 #make sure they are all even

# build an empty 3D matrix to fill with vdW spheres with M_size dimensions
M_fill = np.zeros([int(M_size[0]),int(M_size[1]),int(M_size[2])])

In [68]:
#obtain the coordinates of the C and H atoms in units of grids in the M matrix
pos_H = M_H/d+np.ceil(M_size/2)
pos_C = M_C/d+np.ceil(M_size/2)

In [69]:
# build a 3D grid to represent the elements filled by the C-volume enclosed within
# use geometric relations between x, y, and z for a sphere
# R = np.sqrt(x**2+y**2+z**2)
# z=(+/-)np.sqrt(R**2-x**2+y**2)

#specify the size of the grid to represent C-atoms
C_size = [int(2/d)*2,int(2/d)*2,int(2/d)*2]
#build a matrix to fill with 1s at the position of the C-sphere
C_fill = np.zeros(C_size)
R = vdW_R_C

zs = np.array([])


for i in range(int(np.ceil(R)/d)):
    # Initialize an empty array to store the z-values for this slice
    z_slice = np.array([])

    # Loop over y-coordinates
    for j in range(int(np.ceil(R)/d)):
        # Assign current x and y values
        x = i
        y = j
        # Calculate the positive z-value for the current (x, y) using the sphere equation
        z_plus = +np.sqrt((R/d)**2-x**2-y**2)
        # Calculate the negative z-value for the current (x, y)
        z_minus = -np.sqrt((R/d)**2-x**2-y**2)
        # Append the positive z-value to the z_slice array (for now, only considering the upper half of the sphere)
        z_slice = np.append(z_slice,[z_plus])
        
    # Create a symmetrical slice by appending the flipped z_slice (which represents the lower half of the sphere)
    z_slice = np.append(np.flip(z_slice),z_slice)
    
    # Loop over all x-coordinates in the grid
    for k in range(C_size[0]):
        # Calculate the maximum z index for filling
        z_max = np.ceil(z_slice[k])
        # If the z_max value is positive, fill the grid at the appropriate positions with 1s
        if z_max>0:
            # Fill the upper half of the sphere in the grid
            C_fill[i+int(C_size[0]/2),k,int(C_size[0]/2-z_max-1):int(z_max-1+C_size[0]/2)]=1
            # Fill the lower half of the sphere in the grid (by symmetry)
            C_fill[int(C_size[0]/2)-i,k,int(C_size[0]/2-z_max-1):int(z_max-1+C_size[0]/2)]=1

#compare numerical volume with geometry calculation
print('grid volume: '+str(np.sum(C_fill)*d**3)) 
print('vdW volume: '+str(4*np.pi/3*vdW_R_C**3)) 

/var/folders/5n/dbmlrf856jxdz13g9t5fzxs40000gn/T/ipykernel_50538/363478187.py:25: RuntimeWarning: invalid value encountered in sqrt
  z_plus = +np.sqrt((R/d)**2-x**2-y**2)
/var/folders/5n/dbmlrf856jxdz13g9t5fzxs40000gn/T/ipykernel_50538/363478187.py:27: RuntimeWarning: invalid value encountered in sqrt
  z_minus = -np.sqrt((R/d)**2-x**2-y**2)


grid volume: 20.759652000000003
vdW volume: 20.579526276115534


In [70]:
#repeat the same procedure for an H-atom
H_size = [int(1.5/d)*2,int(1.5/d)*2,int(1.5/d)*2]
H_fill = np.zeros(H_size)
R = vdW_R_H

zs = np.array([])
for i in range(int(np.ceil(R)/d)):
    z_slice = np.array([])
    for j in range(int(np.ceil(R)/d)):

        x = i
        y = j
        z_plus = +np.sqrt((R/d)**2-x**2-y**2)
        z_minus = -np.sqrt((R/d)**2-x**2-y**2)
        z_slice = np.append(z_slice,[z_plus])
    z_slice = np.append(np.flip(z_slice),z_slice)    
    for k in range(H_size[0]):
        z_max = np.ceil(z_slice[k])
        if z_max>0:
            H_fill[i+int(H_size[0]/2),k,int(H_size[0]/2-z_max-1):int(z_max-1+H_size[0]/2)]=1
            H_fill[int(H_size[0]/2)-i,k,int(H_size[0]/2-z_max-1):int(z_max-1+H_size[0]/2)]=1

#compare numerical volume with geometry calculation
print('grid volume: '+str(np.sum(H_fill)*d**3)) 
print('vdW volume: '+str(4*np.pi/3*vdW_R_H**3)) 

/var/folders/5n/dbmlrf856jxdz13g9t5fzxs40000gn/T/ipykernel_50538/3167089378.py:13: RuntimeWarning: invalid value encountered in sqrt
  z_plus = +np.sqrt((R/d)**2-x**2-y**2)
/var/folders/5n/dbmlrf856jxdz13g9t5fzxs40000gn/T/ipykernel_50538/3167089378.py:14: RuntimeWarning: invalid value encountered in sqrt
  z_minus = -np.sqrt((R/d)**2-x**2-y**2)


grid volume: 7.175966000000002
vdW volume: 7.238229473870882


In [71]:
#superimpose spheres by taking the coordinates of each C and H. 
#Add matrix elements +/- dimensions of the atom to the M matrix.
i = 0
for i in range(np.shape(pos_C)[0]):
    M_fill[int(np.ceil(pos_C[i][0])-int(C_size[0]/2)):int(np.ceil(pos_C[i][0])+int(C_size[0]/2)),
       int(np.ceil(pos_C[i][1])-int(C_size[0]/2)): int(np.ceil(pos_C[i][1])+int(C_size[0]/2)),
       int(np.ceil(pos_C[i][2])-int(C_size[0]/2)): int(np.ceil(pos_C[i][2])+int(C_size[0]/2))] += C_fill

i = 0  
for i in range(np.shape(pos_H)[0]):
    M_fill[int(np.ceil(pos_H[i][0])-int(H_size[0]/2)):int(np.ceil(pos_H[i][0])+int(H_size[0]/2)),
       int(np.ceil(pos_H[i][1])-int(H_size[0]/2)): int(np.ceil(pos_H[i][1])+int(H_size[0]/2)),
       int(np.ceil(pos_H[i][2])-int(H_size[0]/2)): int(np.ceil(pos_H[i][2])+int(H_size[0]/2))] += H_fill


In [72]:
#replace all of the values larger than 1 (when they overlap) with 1
M_fill[M_fill>1]=1
#collapse the z-axis of the 3D geomertry down onto the xy plane
vdW_2d = np.sum(M_fill,axis=2)
#replace any non-zero element with 1
vdW_2d[vdW_2d>1] = 1
#count the filled grids and multiply by grid size for the area element to get the vdW area
vdW_area=np.sum(vdW_2d)*d**2
print(vdW_area)

41.208000000000006
